<a href="https://colab.research.google.com/github/liuyao12/imagenette_experiments/blob/master/ResNet_separable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ResNet with depthwise separable convolution

> depthwise (x4) + Ranger + Mish + SA + MaxBlurPool + ResTrick

See summary at https://forums.fast.ai/t/imagenette-imagewoof-leaderboards/45822/47?u=liuyao 

# setup and imports

In [ ]:
# pip install kornia

In [ ]:
pip install git+https://github.com/ayasyrev/model_constructor

In [ ]:
pip install git+https://github.com/ayasyrev/imagenette_experiments

In [3]:
from imagenette_experiments.train_utils import *

In [4]:
from kornia.contrib import MaxBlurPool2d

In [5]:
from fastai.basic_train import *
from fastai.vision import *
# from fastai.script import *
from model_constructor.net import Net, act_fn
from model_constructor.layers import SimpleSelfAttention, ConvLayer

# ResBlock

In [6]:
class NewLayer(nn.Sequential):
    """Basic conv layers block"""
    def __init__(self, ni, nf, ks=3, stride=1,
            act=True,  act_fn=nn.ReLU(inplace=True),
            bn_layer=True, bn_1st=True, zero_bn=False,
            padding=None, bias=False, groups=1, **kwargs):

        if padding==None: padding = ks//2
        if ks==3 and nf!=ni*4:  # to be used for the "stem" of ResNet
          layers = [('Conv3x3', nn.Conv2d(ni, ni*4, 3, stride=1, padding=1, bias=bias, groups=ni)),
                    # ('act_fn', act_fn),
                    ('Conv1x1', nn.Conv2d(ni*4, nf, 1, bias=bias, groups=1))]
        else:
          layers = [('Conv{}x{}'.format(ks,ks), 
                      nn.Conv2d(ni, nf, ks, stride=stride, padding=padding, bias=bias, groups=groups))]

        act_bn = [('act_fn', act_fn)] if act else []
        if bn_layer:
            bn = nn.BatchNorm2d(nf)
            nn.init.constant_(bn.weight, 0. if zero_bn else 1.)
            act_bn += [('bn', bn)]
        if bn_1st: act_bn.reverse()
        layers += act_bn
        super().__init__(OrderedDict(layers))

In [7]:
class NewResBlock(Module):
    def __init__(self, expansion, ni, nh, stride=1,
                 conv_layer=ConvLayer, act_fn=act_fn, bn_1st=True,
                 pool=nn.AvgPool2d(2, ceil_mode=True), sa=False, sym=False, zero_bn=True):
        nf,ni = nh*expansion,ni*expansion
        conv_layer = NewLayer
        self.reduce = noop if stride==1 else pool
        layers  = [(f"conv_0", conv_layer(ni, nh, 3, act_fn=act_fn, bn_1st=bn_1st)),
                   (f"conv_1", conv_layer(nh, nf, 3, zero_bn=zero_bn, act=False, bn_layer=True))
        ] if expansion == 1 else [
                   (f"conv_0", conv_layer(ni, nh, 1, act_fn=act_fn, bn_1st=bn_1st)),
                   # (f"conv_1", conv_layer(nh, nh, 3, act_fn=act_fn, bn_1st=bn_1st)),
                   (f"conv_1", conv_layer(nh, nh*4, 3, groups=nh, act=False, bn_layer=True)),
                   (f"conv_2", conv_layer(nh*4, nf, 1, zero_bn=zero_bn, act=False, bn_1st=bn_1st))
        ]
        if sa: layers.append(('sa', SimpleSelfAttention(nf,ks=1,sym=sym)))
        self.convs = nn.Sequential(OrderedDict(layers))
        self.idconv = noop if ni==nf else conv_layer(ni, nf, 1, act=False, bn_1st=bn_1st)
        self.merge = act_fn

    def forward(self, x):
        o = self.reduce(x)
        return self.merge(self.convs(o) + self.idconv(o))

# Model Constructor

In [8]:
model = Net(c_out=10, layers=[3,6,8,3], expansion=4)
model.block = NewResBlock
# model.conv_layer = NewLayer # for the stem
pool = MaxBlurPool2d(3, True)
model.pool = pool
model.stem_pool = pool
model.stem_sizes = [3,32,64,64]
model.act_fn = Mish()
model.sa = True
res = []

## size=128 (no dw for stem)


In [ ]:
for epochs in [5]*5:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=128, bs=32, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res)) 

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.926666,1.852383,0.400865,0.859506,01:31
1,1.635753,1.545834,0.546704,0.930771,01:31
2,1.448534,1.349256,0.637312,0.949351,01:30
3,1.313903,1.230567,0.693561,0.960041,01:30
4,1.087285,1.060132,0.768898,0.974803,01:30


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


...

epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.947968,1.724592,0.461695,0.890557,01:30
1,1.660980,1.500144,0.562993,0.934080,01:30
2,1.454634,1.318864,0.656656,0.951896,01:29
3,1.319579,1.202859,0.715704,0.960804,01:30
4,1.109657,1.049250,0.781369,0.972512,01:29


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.874006,1.773460,0.433698,0.882158,01:29
1,1.626993,1.531710,0.564266,0.925681,01:30
2,1.446367,1.299589,0.665055,0.953423,01:30
3,1.306742,1.216297,0.705778,0.959786,01:29
4,1.082566,1.053554,0.780606,0.973785,01:30


[0.768898, 0.779842, 0.781115, 0.781369, 0.780606] 0.7783660054206848


In [ ]:
res, '5 mean: {}'.format(sum(res)/5)

([0.7688979506492615,
  0.77984219789505,
  0.7811148166656494,
  0.7813693284988403,
  0.7806057333946228],
 '5 mean: 0.7783660054206848')

In [ ]:
res, '5 mean: {}'.format(sum(res)/5)

([0.8254008889198303,
  0.8086026906967163,
  0.8256554007530212,
  0.8213285803794861,
  0.8139475584030151],
 '5 mean: 0.8189870238304138')

In [ ]:
for epochs in [20]*5:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=128, bs=32, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.949271,1.823427,0.420972,0.866124,01:30
1,1.642933,1.505736,0.563248,0.926953,01:29
2,1.459789,1.360561,0.628913,0.946042,01:30
3,1.337876,1.230123,0.699415,0.954950,01:30
4,1.246346,1.208583,0.699669,0.960041,01:29
5,1.134926,1.115217,0.755409,0.968440,01:29
6,1.088735,1.057945,0.770171,0.974039,01:30
7,1.029235,1.032997,0.784169,0.969458,01:30
8,0.982295,1.000211,0.791550,0.979384,01:30
9,0.964776,0.984725,0.805548,0.973785,01:30


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


...

epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.891150,1.823158,0.428608,0.888267,01:29
1,1.610289,1.543946,0.552049,0.917027,01:30
2,1.446135,1.348680,0.639857,0.946042,01:30
3,1.305553,1.213076,0.707050,0.961822,01:30
4,1.226971,1.135819,0.743446,0.966149,01:30
5,1.135491,1.101483,0.752609,0.969967,01:30
6,1.075037,1.073542,0.769152,0.968694,01:30
7,1.015514,1.042395,0.777552,0.972767,01:30
8,0.985679,1.010991,0.790532,0.975566,01:30
9,0.960062,1.001864,0.797404,0.976584,01:30


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.891569,1.710517,0.453041,0.897684,01:30
1,1.643553,1.443666,0.594808,0.944261,01:30
2,1.431929,1.319665,0.651565,0.951387,01:30
3,1.315495,1.235722,0.687961,0.961313,01:30
4,1.192190,1.121433,0.747773,0.963858,01:30
5,1.138651,1.067773,0.769152,0.971494,01:30
6,1.077378,1.048082,0.780860,0.974548,01:29
7,1.036416,1.075319,0.769152,0.971749,01:30
8,0.976910,0.991011,0.804021,0.974803,01:30
9,0.939330,1.024645,0.792314,0.975312,01:30


[0.768898, 0.779842, 0.781115, 0.781369, 0.780606, 0.856197, 0.861033, 0.859252, 0.859252, 0.861288] 0.818885213136673


In [ ]:
res[5:], sum(res[5:])/5

([0.8561974763870239,
  0.8610333204269409,
  0.8592517375946045,
  0.8592517375946045,
  0.8612878322601318],
 0.8594044208526611)

In [ ]:
sum(res[5:])/5

0.8662255048751831

In [ ]:
res = [0.861542, 0.859252, 0.859252, 0.863324, 0.864851]
sum(res)/5

0.8616442000000001

In [ ]:
for epochs in [20]:
    mixup=0 if epochs<=20 else 0.2
    model.conv_layer = NewLayer # for the stem
    learn = get_learn(model=model, size=128, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.829201,1.721854,0.448969,0.902011,03:59
1,1.565073,1.483950,0.569611,0.931280,03:58
2,1.368742,1.273024,0.679308,0.961313,03:58
3,1.269593,1.182214,0.726139,0.967676,03:58
4,1.183783,1.200593,0.733011,0.962586,03:58
5,1.090689,1.081747,0.778315,0.972003,03:58
6,1.036526,1.003656,0.811657,0.978621,03:58
7,1.004153,0.970588,0.820056,0.981929,03:58
8,0.970028,0.934731,0.839145,0.981420,03:58
9,0.949998,0.927825,0.837618,0.981420,04:00


Buffered data was truncated after reaching the output size limit.

In [ ]:
for i in range(20):
    print('epoch {} {}'.format(i, learn.recorder.metrics[i][0].item()))

epoch 0 0.4489692151546478
epoch 1 0.569610595703125
epoch 2 0.6793076992034912
epoch 3 0.7261389493942261
epoch 4 0.7330109477043152
epoch 5 0.7783150672912598
epoch 6 0.8116568922996521
epoch 7 0.8200560212135315
epoch 8 0.8391448259353638
epoch 9 0.8376176953315735
epoch 10 0.8396538496017456
epoch 11 0.8452532291412354
epoch 12 0.8307457566261292
epoch 13 0.8488165140151978
epoch 14 0.8282005786895752
epoch 15 0.86077880859375
epoch 16 0.870450496673584
epoch 17 0.8808857202529907
epoch 18 0.8836854100227356
epoch 19 0.8867396116256714


In [ ]:
for epochs in [80]:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=128, bs=32, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,2.077635,1.882404,0.382031,0.861288,01:30
1,1.866927,1.771457,0.425554,0.897429,01:30
2,1.684625,1.410609,0.614915,0.944515,01:30
3,1.549005,1.299853,0.666327,0.948078,01:30
4,1.483266,1.194461,0.711122,0.961059,01:30
5,1.416350,1.225254,0.691779,0.956732,01:30
6,1.380229,1.093198,0.753627,0.970730,01:31
7,1.315334,1.053309,0.772970,0.975821,01:30
8,1.280437,1.040477,0.779333,0.974039,01:30
9,1.234804,1.018804,0.791295,0.973276,01:31


[0.881395] 0.8813947439193726


## size=128 (dw for stem as well)

`batch_size=16` due to GPU memory constraint

In [ ]:
# depthwise: stem x4, body x4
model.conv_layer = NewLayer
for epochs in [5]*5:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=128, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res)) 

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.848065,1.802390,0.444388,0.879613,03:58
1,1.542165,1.417354,0.610079,0.936625,03:58
2,1.346171,1.221739,0.704759,0.963349,03:59
3,1.182617,1.138943,0.752100,0.965386,03:58
4,1.025726,0.981215,0.825401,0.978621,03:58


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


...

epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.782196,1.785809,0.459913,0.881904,03:58
1,1.542367,1.375912,0.623568,0.952151,03:57
2,1.372330,1.222670,0.702723,0.956986,03:58
3,1.208639,1.136777,0.748282,0.970221,03:58
4,1.026186,0.984933,0.821329,0.979384,03:58


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.843992,1.767481,0.447442,0.900229,03:58
1,1.588587,1.503676,0.560957,0.937643,03:57
2,1.368809,1.249841,0.686689,0.960804,03:57
3,1.260239,1.132880,0.752609,0.973021,03:57
4,1.023229,1.008679,0.813948,0.978366,03:58


[0.825401, 0.808603, 0.825655, 0.821329, 0.813948] 0.8189870238304138


In [ ]:
# depthwise: stem x4, body x4 (no act)
model.conv_layer = NewLayer
for epochs in [5]*5:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=128, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res)) 

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.832179,1.682135,0.476457,0.905828,03:44
1,1.573933,1.439008,0.589972,0.937643,03:44
2,1.349251,1.279347,0.681089,0.956986,03:43
3,1.234181,1.135676,0.750064,0.967676,03:43
4,1.025385,0.992665,0.817002,0.979130,03:43


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


...

epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.844609,1.771936,0.442352,0.886485,03:43
1,1.594411,1.445501,0.586409,0.946551,03:42
2,1.334774,1.239162,0.701705,0.964368,03:43
3,1.235657,1.160112,0.745228,0.966913,03:43
4,1.005870,0.981502,0.824637,0.980657,03:43


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.899922,1.722196,0.446933,0.886994,03:43
1,1.600595,1.464100,0.580300,0.942734,03:43
2,1.414322,1.308025,0.662764,0.955968,03:42
3,1.250982,1.152586,0.745991,0.967422,03:42
4,1.045228,1.009484,0.807839,0.980657,03:42


[0.817002, 0.809112, 0.821329, 0.824637, 0.807839] 0.8159837007522583


In [ ]:
# depthwise: stem x6, body x4
model.conv_layer = NewLayer # for the stem
for epochs in [5]*5:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=128, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res)) 

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.847798,1.714627,0.462713,0.895902,04:10
1,1.556042,1.433400,0.606261,0.944261,04:08
2,1.385322,1.258171,0.672945,0.962331,04:08
3,1.218732,1.131767,0.746500,0.972258,04:08
4,1.005931,0.982060,0.821074,0.982438,04:09


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


...

epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.819326,1.689506,0.473148,0.907356,04:08
1,1.549364,1.398404,0.620260,0.940952,04:07
2,1.321471,1.295579,0.680326,0.954950,04:08
3,1.185924,1.088459,0.779842,0.972512,04:08
4,1.023251,0.989862,0.820565,0.982693,04:08


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.824531,1.653337,0.495800,0.919827,04:08
1,1.529207,1.348908,0.633749,0.951387,04:09
2,1.390288,1.277264,0.675490,0.951896,04:08
3,1.227537,1.152907,0.754136,0.970221,04:07
4,0.998385,0.994456,0.824383,0.981675,04:07


[0.821074, 0.816238, 0.813948, 0.820565, 0.824383] 0.8192415356636047


In [ ]:
model.conv_layer = NewLayer
for epochs in [20]*5:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=128, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.836957,1.713554,0.472639,0.895902,03:43
1,1.563930,1.473249,0.587681,0.933825,03:43
2,1.394160,1.278993,0.683635,0.959023,03:43
3,1.274994,1.237810,0.707814,0.963858,03:43
4,1.172646,1.121431,0.744210,0.970985,03:43
5,1.079930,1.043805,0.781624,0.977348,03:44
6,1.065637,1.079634,0.767880,0.971749,03:44
7,1.010309,0.988310,0.819038,0.981166,03:45
8,0.964538,0.948232,0.829728,0.978875,03:45
9,0.929820,0.966749,0.829473,0.977602,03:45


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


...

epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.880377,1.774616,0.446933,0.886231,03:43
1,1.598080,1.484488,0.565029,0.935353,03:42
2,1.392710,1.284517,0.680835,0.959277,03:43
3,1.309178,1.181765,0.731484,0.964622,03:42
4,1.193256,1.096960,0.769661,0.970730,03:42
5,1.135673,1.063450,0.779842,0.974548,03:43
6,1.040701,1.016662,0.796386,0.974803,03:43
7,1.019129,1.011088,0.806821,0.978366,03:43
8,0.962847,0.973563,0.825401,0.980148,03:43
9,0.951495,0.952818,0.826419,0.981166,03:43


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.865880,1.661513,0.471621,0.910919,03:42
1,1.583696,1.518348,0.572919,0.926953,03:42
2,1.375456,1.318276,0.673199,0.959023,03:42
3,1.293896,1.175204,0.724866,0.969712,03:42
4,1.158185,1.085072,0.773988,0.973785,03:42
5,1.119079,1.095060,0.777552,0.974803,03:42
6,1.080353,1.025463,0.801222,0.977093,03:42
7,0.991430,0.999090,0.813439,0.977093,03:43
8,0.952309,0.948992,0.832273,0.982693,03:42
9,0.936540,0.975197,0.821838,0.977857,03:42


[0.888521, 0.887249, 0.887503, 0.891066, 0.884449] 0.8877576947212219


In [ ]:
model.conv_layer = NewLayer
for epochs in [80]:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=128, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.975242,1.788123,0.419445,0.889030,03:58
1,1.780006,1.442986,0.610079,0.939170,03:58
2,1.581596,1.288297,0.682871,0.955968,03:58
3,1.462342,1.226046,0.705014,0.961568,03:58
4,1.430518,1.138328,0.751336,0.965640,03:58
5,1.321730,1.076725,0.774243,0.970730,03:58
6,1.313482,1.065050,0.774497,0.975566,03:58
7,1.273843,1.010651,0.803512,0.974548,03:58
8,1.259136,0.976015,0.822601,0.977093,03:58
9,1.200118,0.937788,0.834054,0.979130,03:58


[0.88674, 0.89692] 0.8918299674987793


In [ ]:
# depthwise: stem x4, body x4 (no act)
model.conv_layer = NewLayer
for epochs in [80]:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=128, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,2.016921,1.872833,0.381267,0.842708,03:43
1,1.819129,1.557309,0.538050,0.928481,03:43
2,1.634017,1.360916,0.644184,0.951387,03:43
3,1.511502,1.198855,0.720285,0.964368,03:43
4,1.445111,1.143026,0.747518,0.967422,03:43
5,1.361953,1.132886,0.753881,0.970476,03:43
6,1.335636,1.013129,0.802494,0.978366,03:43
7,1.297703,1.039696,0.793332,0.973785,03:43
8,1.263235,0.973626,0.821838,0.979384,03:43
9,1.216685,0.954794,0.826419,0.978875,03:43


[0.817002, 0.809112, 0.821329, 0.824637, 0.807839, 0.902265] 0.830363949139913


In [ ]:
# depthwise: stem x4, body x4 (no act)
model.conv_layer = NewLayer
for epochs in [200]:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=128, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.929992,1.781422,0.447951,0.880631,03:43
1,1.704752,1.500633,0.577246,0.932298,03:43
2,1.574633,1.278493,0.669127,0.959532,03:43
3,1.478306,1.202635,0.712650,0.961313,03:43
4,1.395416,1.104136,0.757699,0.972003,03:43
5,1.355051,1.059350,0.779842,0.977602,03:43
6,1.306909,1.057403,0.777552,0.976839,03:44
7,1.252242,0.990524,0.813184,0.976839,03:43
8,1.247131,1.012151,0.807330,0.977093,03:43
9,1.178888,0.969017,0.826164,0.978112,03:43


[0.817002, 0.809112, 0.821329, 0.824637, 0.807839, 0.902265, 0.907101] 0.84132638999394


## size=192

In [ ]:
for epochs in [5]*5:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=192, bs=32, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.852821,1.733783,0.456350,0.890303,02:22
1,1.582649,1.475512,0.574446,0.934843,02:22
2,1.386568,1.275406,0.674981,0.954950,02:23
3,1.230152,1.134809,0.741919,0.964113,02:23
4,1.015104,0.979367,0.820311,0.980148,02:23


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


...

epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.902325,1.814900,0.439043,0.876559,02:23
1,1.623413,1.459575,0.578773,0.935862,02:23
2,1.431342,1.316085,0.652074,0.953169,02:23
3,1.228662,1.166410,0.736320,0.970476,02:23
4,1.043673,1.010692,0.804785,0.979384,02:23


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.830721,1.724038,0.471876,0.904301,02:23
1,1.564874,1.366736,0.630949,0.947569,02:23
2,1.366136,1.275826,0.679053,0.954441,02:23
3,1.216263,1.147997,0.741410,0.968185,02:23
4,1.018347,0.993846,0.811148,0.978621,02:23


[0.820311, 0.812166, 0.809112, 0.804785, 0.811148] 0.8115041971206665


In [ ]:
for epochs in [20]*5:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=192, bs=32, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.883860,1.728872,0.460932,0.900229,02:20
1,1.614101,1.489058,0.570883,0.935353,02:20
2,1.414853,1.266919,0.687707,0.958514,02:20
3,1.244259,1.165708,0.738356,0.963604,02:20
4,1.170540,1.072613,0.771189,0.969712,02:20
5,1.084809,1.092608,0.768389,0.976330,02:20
6,1.013404,0.989086,0.806567,0.980911,02:21
7,0.972647,0.980479,0.810130,0.980148,02:20
8,0.928764,0.963534,0.824128,0.980911,02:21
9,0.903548,0.918316,0.840926,0.980657,02:21


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


...

epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.883659,1.792554,0.432680,0.882667,02:21
1,1.625097,1.491719,0.579791,0.934334,02:21
2,1.405370,1.315328,0.655129,0.954950,02:21
3,1.247899,1.287617,0.676253,0.956477,02:21
4,1.182906,1.094138,0.771443,0.973785,02:21
5,1.109006,1.073522,0.777297,0.976075,02:21
6,1.022097,1.022035,0.786714,0.975566,02:22
7,0.986480,0.967262,0.817002,0.979384,02:21
8,0.925861,0.942716,0.834818,0.978875,02:22
9,0.907570,0.954938,0.822347,0.979130,02:22


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.869943,1.766961,0.444897,0.866633,02:23
1,1.569097,1.463634,0.594044,0.937134,02:24
2,1.412598,1.270381,0.677526,0.960041,02:22
3,1.247438,1.169785,0.737083,0.964368,02:22
4,1.151413,1.078368,0.780351,0.974294,02:22
5,1.067297,1.054951,0.781369,0.975566,02:23
6,1.025227,1.005551,0.806567,0.975821,02:24
7,0.970661,0.948971,0.823874,0.983456,02:24
8,0.931109,0.940674,0.828201,0.982693,02:24
9,0.895758,0.925844,0.838381,0.983202,02:23


[0.883685, 0.883431, 0.883176, 0.884958, 0.883431] 0.8837363243103027


In [ ]:
for epochs in [80]:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=192, bs=32, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.999242,1.748933,0.443624,0.887503,02:22
1,1.800613,1.503827,0.581064,0.934080,02:21
2,1.624292,1.330687,0.652074,0.950624,02:22
3,1.485650,1.184846,0.725630,0.963349,02:22
4,1.402756,1.103704,0.762280,0.969203,02:22
5,1.351776,1.060633,0.778315,0.975566,02:22
6,1.310300,1.007548,0.803003,0.980657,02:22
7,1.258783,0.978774,0.811911,0.978112,02:22
8,1.224756,0.955228,0.823365,0.977348,02:22
9,1.194061,0.945081,0.825146,0.981166,02:22


[0.898956] 0.8989564776420593


In [ ]:
res = []
for epochs in [200]:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=192, bs=32, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.977234,1.746857,0.447697,0.885722,02:24
1,1.754137,1.551928,0.549758,0.922372,02:24
2,1.631945,1.349416,0.628913,0.951642,02:24
3,1.482694,1.198492,0.719776,0.960550,02:24
4,1.424144,1.113130,0.750064,0.967931,02:24
5,1.338592,1.033281,0.787987,0.975312,02:23
6,1.288472,1.011857,0.801985,0.975312,02:23
7,1.253425,0.968262,0.819292,0.980911,02:23
8,1.203076,0.962628,0.819801,0.979384,02:22
9,1.181931,0.918316,0.840672,0.982947,02:23


[0.903283] 0.9032832980155945


## size=192 (dw for stem as well)

batch_size=16 (no act no bn)

In [ ]:
model.conv_layer = NewLayer
for epochs in [5]*5:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=192, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res)) 

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.788149,1.616220,0.513617,0.908883,07:11
1,1.515203,1.494594,0.580809,0.941206,07:11
2,1.383367,1.256758,0.694833,0.965640,07:10
3,1.247741,1.128057,0.765589,0.967676,07:10
4,1.036198,0.989479,0.823619,0.978112,07:10


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


...

epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.847069,1.735877,0.436243,0.900484,07:10
1,1.529914,1.421316,0.602443,0.950624,07:10
2,1.454184,1.304343,0.662510,0.959786,07:11
3,1.264978,1.172860,0.740901,0.970221,07:11
4,1.099680,1.009021,0.812166,0.978621,07:11


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.847264,1.726892,0.453296,0.894121,07:11
1,1.509220,1.451855,0.596335,0.942734,07:11
2,1.387435,1.257013,0.699415,0.963095,07:11
3,1.215625,1.137987,0.754136,0.970730,07:11
4,1.033624,1.004465,0.818529,0.979893,07:11


[0.806567, 0.806821, 0.817002, 0.805803, 0.806058, 0.823619, 0.816747, 0.817002, 0.812166, 0.818529] 0.8130312919616699


In [ ]:
res[5:], sum(res[5:])/5

([0.8236192464828491,
  0.81674724817276,
  0.8170017600059509,
  0.8121659159660339,
  0.8185288906097412],
 0.8176126122474671)

In [9]:
model.conv_layer = NewLayer
for epochs in [5]:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=192, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/u

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.857717,1.766861,0.457368,0.892084,06:47
1,1.621690,1.490592,0.572410,0.943497,06:47
2,1.452886,1.303712,0.672945,0.955714,06:47
3,1.270742,1.205157,0.727412,0.969203,06:47
4,1.017069,1.003154,0.815984,0.979893,06:47


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value)


[0.815984] 0.8159837126731873


In [10]:
model.conv_layer = NewLayer
for epochs in [20]:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=192, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.920210,1.934003,0.389158,0.851871,07:06
1,1.607510,1.525460,0.569865,0.929753,07:05
2,1.447699,1.379700,0.630695,0.954950,07:05
3,1.278898,1.183866,0.737338,0.966913,07:06
4,1.197482,1.108146,0.772207,0.966404,07:06
5,1.154319,1.077412,0.783151,0.976075,07:06
6,1.101685,1.048601,0.788750,0.976839,07:06
7,0.999846,0.973895,0.828964,0.979893,07:06
8,0.957927,0.952103,0.842963,0.981420,07:06
9,0.908149,0.937676,0.838381,0.984474,07:06


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value)


[0.900993] 0.9009926319122314


In [28]:
print([round(x, 6) for x in res], sum(res)/len(res))

[0.882413, 0.884703] 0.8835581541061401


In [10]:
model.conv_layer = NewLayer
for epochs in [80]:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=192, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res)) 

data path   /root/.fastai/data/imagewoof2


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.987625,1.807268,0.436752,0.878595,06:47
1,1.749539,1.558918,0.551794,0.928481,06:47
2,1.648091,1.341580,0.644693,0.954696,06:47
3,1.531452,1.221247,0.703741,0.969458,06:47
4,1.405284,1.130693,0.756936,0.968185,06:47
5,1.388755,1.066229,0.778570,0.974294,06:47
6,1.322789,1.037073,0.798422,0.976330,06:47
7,1.325094,1.039731,0.792823,0.976330,06:48
8,1.220204,0.981795,0.820820,0.975057,06:48
9,1.222531,0.952349,0.827437,0.981420,06:48


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "


Buffered data was truncated after reaching the output size limit.

In [11]:
res

[0.8159837126731873, 0.909646213054657]

In [12]:
for i in range(80):
    print('epoch {} {}'.format(i, learn.recorder.metrics[i][0].item()))

epoch 0 0.4367523491382599
epoch 1 0.5517943501472473
epoch 2 0.6446933150291443
epoch 3 0.7037414312362671
epoch 4 0.7569355964660645
epoch 5 0.7785696387290955
epoch 6 0.7984219789505005
epoch 7 0.7928225994110107
epoch 8 0.8208195567131042
epoch 9 0.8274369835853577
epoch 10 0.8350725173950195
epoch 11 0.8419445157051086
epoch 12 0.8536523580551147
epoch 13 0.8472893834114075
epoch 14 0.8505980968475342
epoch 15 0.8676508069038391
epoch 16 0.8699414730072021
epoch 17 0.877577006816864
epoch 18 0.8712140321731567
epoch 19 0.8791040778160095
epoch 20 0.8648511171340942
epoch 21 0.8750318288803101
epoch 22 0.8724866509437561
epoch 23 0.8671417832374573
epoch 24 0.8724866509437561
epoch 25 0.8778315186500549
epoch 26 0.8847034573554993
epoch 27 0.8701959848403931
epoch 28 0.8821583390235901
epoch 29 0.8765588998794556
epoch 30 0.8768134117126465
epoch 31 0.8808857202529907
epoch 32 0.8765588998794556
epoch 33 0.8877577185630798
epoch 34 0.8816492557525635
epoch 35 0.8872486352920532
epo

In [12]:
model.conv_layer = NewLayer
for epochs in [80]:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=192, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/u

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,2.058254,1.851818,0.402392,0.876304,05:16
1,1.891856,1.724944,0.463222,0.896157,05:15
2,1.746456,1.469180,0.610842,0.947569,05:15
3,1.638097,1.429703,0.609824,0.953169,05:15
4,1.537492,1.241984,0.693815,0.964113,05:15
5,1.528490,1.203215,0.720031,0.968185,05:15
6,1.441906,1.175080,0.734284,0.969203,05:15
7,1.407579,1.122729,0.757954,0.971494,05:16
8,1.390933,1.114469,0.761771,0.970730,05:15
9,1.323530,1.035686,0.801222,0.974039,05:15


[0.781115, 0.909392] 0.8452532589435577


In [12]:
res

[0.9009926319122314, 0.9132094383239746]

In [13]:
for i in range(80):
    print('epoch {} {}'.format(i, learn.recorder.metrics[i][0].item()))

epoch 0 0.41842707991600037
epoch 1 0.4874013662338257
epoch 2 0.6355306506156921
epoch 3 0.6953423023223877
epoch 4 0.7490455508232117
epoch 5 0.782387375831604
epoch 6 0.7508271932601929
epoch 7 0.8002036213874817
epoch 8 0.8348180055618286
epoch 9 0.8363451361656189
epoch 10 0.8353270292282104
epoch 11 0.8386358022689819
epoch 12 0.8472893834114075
epoch 13 0.8485620021820068
epoch 14 0.86077880859375
epoch 15 0.8735046982765198
epoch 16 0.8605242967605591
epoch 17 0.8701959848403931
epoch 18 0.8735046982765198
epoch 19 0.872741162776947
epoch 20 0.872741162776947
epoch 21 0.8729956746101379
epoch 22 0.8819037675857544
epoch 23 0.8768134117126465
epoch 24 0.8735046982765198
epoch 25 0.8770679831504822
epoch 26 0.8747773170471191
epoch 27 0.8778315186500549
epoch 28 0.8714685440063477
epoch 29 0.8813947439193726
epoch 30 0.8765588998794556
epoch 31 0.8785950541496277
epoch 32 0.877577006816864
epoch 33 0.8852125406265259
epoch 34 0.8783405423164368
epoch 35 0.8819037675857544
epoch 3

In [21]:
model.conv_layer = NewLayer
for epochs in [80]:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=192, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/u

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,2.000776,1.838220,0.397811,0.874268,07:07
1,1.815400,1.603967,0.520743,0.924917,07:07
2,1.621137,1.362221,0.647238,0.947824,07:07
3,1.530266,1.202703,0.719267,0.964113,07:07
4,1.441653,1.148288,0.744973,0.967167,07:07
5,1.382780,1.093323,0.774497,0.974548,07:07
6,1.311637,1.054789,0.785696,0.975821,07:07
7,1.290228,0.999025,0.815984,0.977857,07:07
8,1.259167,0.962138,0.830237,0.975566,07:07
9,1.238394,0.947319,0.837872,0.982184,07:06


Buffered data was truncated after reaching the output size limit.

In [22]:
for i in range(80):
    print('epoch {} {}'.format(i, learn.recorder.metrics[i][0].item()))

epoch 0 0.3978111445903778
epoch 1 0.520743191242218
epoch 2 0.6472384929656982
epoch 3 0.7192670106887817
epoch 4 0.7449733018875122
epoch 5 0.7744973301887512
epoch 6 0.7856960892677307
epoch 7 0.8159837126731873
epoch 8 0.8302366733551025
epoch 9 0.8378722071647644
epoch 10 0.8393993377685547
epoch 11 0.8477984070777893
epoch 12 0.8462713360786438
epoch 13 0.8457622528076172
epoch 14 0.8551794290542603
epoch 15 0.8521252274513245
epoch 16 0.8666327595710754
epoch 17 0.8569610714912415
epoch 18 0.86790531873703
epoch 19 0.8826673626899719
epoch 20 0.8724866509437561
epoch 21 0.8673962950706482
epoch 22 0.8732501864433289
epoch 23 0.8689233660697937
epoch 24 0.882412850856781
epoch 25 0.8864850997924805
epoch 26 0.8783405423164368
epoch 27 0.8780860304832458
epoch 28 0.8869941234588623
epoch 29 0.8836854100227356
epoch 30 0.8778315186500549
epoch 31 0.8836854100227356
epoch 32 0.8839399218559265
epoch 33 0.8732501864433289
epoch 34 0.8869941234588623
epoch 35 0.8829218745231628
epoch 

## size=256

In [ ]:
for epochs in [5]*5:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=256, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2
Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.878585,1.697816,0.466785,0.900229,03:48
1,1.603976,1.470119,0.578519,0.940443,03:48
2,1.401998,1.290956,0.677526,0.951133,03:48
3,1.250543,1.188039,0.739628,0.970221,03:48
4,1.047161,0.984337,0.823619,0.981929,03:48


data path   /root/.fastai/data/imagewoof2
Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.872547,1.705041,0.465767,0.904047,03:48
1,1.608362,1.400562,0.618478,0.940188,03:48
2,1.392648,1.329650,0.653347,0.952660,03:48
3,1.264524,1.124228,0.766098,0.968185,03:47
4,1.019082,0.976503,0.823365,0.979384,03:48


data path   /root/.fastai/data/imagewoof2


...

epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.896510,1.797168,0.429117,0.877068,03:48
1,1.558683,1.386114,0.631204,0.948842,03:47
2,1.404002,1.297136,0.665055,0.964622,03:48
3,1.228450,1.102848,0.764826,0.974548,03:48
4,1.022271,0.983633,0.822856,0.977857,03:48


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.887786,1.733244,0.456350,0.897175,03:48
1,1.601669,1.453238,0.598880,0.940443,03:48
2,1.413110,1.227790,0.705778,0.966149,03:48
3,1.245979,1.115398,0.763044,0.975057,03:48
4,1.034911,0.985657,0.820820,0.981675,03:49


[0.823619, 0.823365, 0.82591, 0.822856, 0.82082] 0.82331383228302


In [ ]:
for epochs in [5]*5:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=256, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.886319,1.874273,0.393993,0.885213,03:33
1,1.590686,1.523450,0.540341,0.929244,03:33
2,1.398079,1.302810,0.655638,0.948333,03:33
3,1.226579,1.154759,0.738101,0.971494,03:33
4,1.045482,0.989257,0.823619,0.981420,03:34


data path   /root/.fastai/data/imagewoof2
Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.884851,1.745729,0.440570,0.887503,03:33
1,1.534766,1.430336,0.609570,0.948333,03:34
2,1.392482,1.249609,0.695851,0.962586,03:34
3,1.264362,1.139566,0.752609,0.971240,03:34
4,1.034837,0.998294,0.818529,0.981929,03:34


...

epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.939716,1.808947,0.423263,0.887758,03:36
1,1.662655,1.527018,0.549249,0.932044,03:36
2,1.437887,1.317161,0.655383,0.958514,03:35
3,1.290576,1.201045,0.726648,0.967422,03:35
4,1.069005,1.015105,0.810639,0.978366,03:35


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.893374,1.754322,0.432171,0.879868,03:35
1,1.543962,1.455444,0.590227,0.941970,03:35
2,1.378485,1.308274,0.654110,0.956986,03:34
3,1.246718,1.141537,0.746755,0.968949,03:34
4,1.030265,0.991779,0.822092,0.979130,03:34


[0.893612, 0.899211, 0.894375, 0.896666, 0.893103, 0.823619, 0.818529, 0.818783, 0.810639, 0.822092] 0.8570628643035889


In [ ]:
for epochs in [20]*5:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=256, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2
Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.911908,1.816824,0.419445,0.888776,03:32
1,1.635836,1.501151,0.565793,0.938661,03:32
2,1.406875,1.286252,0.683380,0.958514,03:33
3,1.308295,1.178766,0.730211,0.970730,03:32
4,1.173939,1.097599,0.764062,0.972512,03:32
5,1.109023,1.034590,0.797913,0.977602,03:32
6,1.101197,1.019133,0.800458,0.974548,03:33
7,1.018536,1.003955,0.804530,0.975821,03:33
8,0.952619,0.984662,0.819038,0.977093,03:33
9,0.991934,0.926092,0.838636,0.983202,03:32


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.913942,1.931084,0.364978,0.860270,03:33
1,1.595795,1.598458,0.519980,0.914737,03:34
2,1.403894,1.252481,0.685925,0.963095,03:34
3,1.254615,1.230131,0.709341,0.966149,03:34
4,1.167179,1.108323,0.756681,0.968185,03:34
5,1.139536,1.078271,0.773988,0.976839,03:34
6,1.062750,1.006647,0.811911,0.976330,03:34
7,1.060679,0.986362,0.818274,0.976075,03:33
8,0.965730,0.956258,0.838127,0.982438,03:34
9,0.937918,0.949168,0.837363,0.983711,03:34


...

epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.923993,1.774940,0.425554,0.880631,03:34
1,1.632135,1.510621,0.557139,0.926444,03:34
2,1.427344,1.322677,0.651820,0.956986,03:34
3,1.270684,1.178054,0.724103,0.969712,03:33
4,1.186677,1.080862,0.776533,0.973530,03:33
5,1.100333,1.024524,0.798676,0.978366,03:33
6,1.082254,1.037213,0.791550,0.978875,03:33
7,1.027036,0.980826,0.820820,0.978112,03:33
8,0.975796,0.969936,0.820565,0.981166,03:33
9,0.968014,0.916382,0.849071,0.981420,03:33


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.914879,1.780073,0.420718,0.875795,03:34
1,1.603976,1.445524,0.595571,0.939425,03:34
2,1.432504,1.283728,0.680326,0.953169,03:34
3,1.260722,1.193832,0.727157,0.963604,03:35
4,1.195908,1.081539,0.780351,0.975312,03:35
5,1.117886,1.054004,0.795368,0.975821,03:34
6,1.097234,1.016342,0.801476,0.978366,03:34
7,1.014749,0.993898,0.830491,0.978366,03:34
8,0.991418,0.964487,0.827946,0.979893,03:34
9,0.968139,0.940190,0.836345,0.982184,03:34


[0.893612, 0.899211, 0.894375, 0.896666, 0.893103] 0.8953932285308838


In [ ]:
for epochs in [80]:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=256, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2
Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,2.032105,1.872854,0.367778,0.869941,03:49
1,1.793629,1.559720,0.529397,0.926444,03:49
2,1.631899,1.346997,0.645457,0.950624,03:49
3,1.527347,1.218183,0.706541,0.967167,03:49
4,1.458608,1.114160,0.764826,0.968440,03:48
5,1.377961,1.070757,0.780351,0.973785,03:49
6,1.340125,1.026627,0.799440,0.970985,03:49
7,1.296936,0.996647,0.810893,0.979130,03:49
8,1.249376,0.944632,0.827946,0.980657,03:49
9,1.217005,0.930064,0.840417,0.980402,03:49


[0.909392] 0.9093917012214661
